In [1]:
import os
import sys
%matplotlib inline

import numpy as np
import pandas as pd
import ipywidgets
from scipy.optimize import Bounds
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from src import data
from src import efficient_frontier
from src import plot
from src import portfolio
from src import capm
from src import backtesting
import numpy as np
import pandas as pd
import ipywidgets
from scipy.optimize import Bounds
import datetime
from dateutil import rrule
import yfinance as yf
import warnings

warnings.filterwarnings("ignore", category=FutureWarning, module='pandas')

In [2]:
x = pd.Timestamp(np.datetime64('2000-01-01'))
x1 = pd.Timestamp(np.datetime64('2022-01-01'))

In [3]:
weights = np.array([1/3,1/3,1/3])
min_esg_score = 1100
max_esg_score = 2000
df = pd.read_excel('../data/ESG_DATA_S&P500.xlsx')
dates = [x,x1]

start_year = dates[0]
end_year = dates[1]

Bounds1 = Bounds(0,1) #How willing we are to go short and to invest in one particular stock
Wanted_return = 0.20 #Only used when using the wanted_return constraint
maximum_risk = 0.10 #Either used when using maximum risk constraint or cmle portfolio
sharpe_type = "No_extra_constraint"  # rename to constraint, can be either Wanted_return or Maximum_risk or No_extra_constraint
risk_free_rate = 0.01
covariance_window = 5 #has to be in years
rebalancing_freq = 'monthly'

strategy1 = {'df': df,
 'weights': weights,
 'min_esg_score': min_esg_score,
 'max_esg_score': max_esg_score,
 'Bounds1': Bounds1,
 'sharpe_type': sharpe_type,
 'Wanted_return': Wanted_return,
 'maximum_risk': maximum_risk,
 'rebalancing_freq': rebalancing_freq,
 'risk_free_rate': risk_free_rate,
 'start_year' : start_year,
 'end_year' : end_year}


In [4]:
data5 = backtesting.backtesting(strategy1, 'yearly', 1, datetime.datetime(2015,1,1),datetime.datetime(2022,1,1),10,'SPY')

Min. Risk = 6.749% => Return: 0.974%  Sharpe Ratio = 0.14
Max. Sharpe Ratio = 0.21 => Return: 2.12%  Risk: 9.941%
Excpected return on investment is 115.01056405516898%
Min. Risk = 6.974% => Return: 1.327%  Sharpe Ratio = 0.19
Max. Sharpe Ratio = 0.27 => Return: 2.61%  Risk: 9.774%
Excpected return on investment is 117.55195125133619%
Min. Risk = 7.007% => Return: 1.614%  Sharpe Ratio = 0.23
Max. Sharpe Ratio = 0.30 => Return: 2.74%  Risk: 9.122%
Excpected return on investment is 115.32434806635763%
Min. Risk = 6.076% => Return: 1.471%  Sharpe Ratio = 0.24
Max. Sharpe Ratio = 0.29 => Return: 2.18%  Risk: 7.401%
Excpected return on investment is 111.2829220717763%
Min. Risk = 6.156% => Return: 2.202%  Sharpe Ratio = 0.36
Max. Sharpe Ratio = 0.39 => Return: 2.64%  Risk: 6.744%
Excpected return on investment is 110.0142007909844%
Min. Risk = 5.504% => Return: 1.981%  Sharpe Ratio = 0.36
Max. Sharpe Ratio = 0.38 => Return: 2.20%  Risk: 5.805%
Excpected return on investment is 109.9424185547

KeyboardInterrupt: 

In [ ]:
#len(data5)
#print(data5[7],data5[6],data5[5])

plot.compare_returns1(data5[7],data5[5],data5[6])